In [48]:
from sage.coding.guruswami_sudan.interpolation import gs_interpolation_lee_osullivan
from sage.coding.guruswami_sudan.gs_decoder import alekhnovich_root_finder, roth_ruckenstein_root_finder

class GRS:
    def __init__(self,q,n,k, K, g, y):
        self.q = q
        self.n = n
        self.k = k
        self.d = n - k + 1
        self.t = floor((self.d - 1)/2.0)
        self.K = K
        self.g = g
        self.x = [g^i for i in range(n-1)]
        self.x.append(0)
        self.y = [y[i] for i in range(n)]
        
    def encode(self,M):
        return vector(K,[sum([self.y[j]*self.x[j]^(self.k-i-1)*M[i] for i in range(self.k)]) for j in range(n)])
        
    def distance_min(self,K):
        info_space = VectorSpace(K,self.k)
        w_min = self.n

        for M in info_space:
            w_tmp = self.encode(M).hamming_weight()
            if w_tmp > 0 and w_tmp<w_min:
                w_min = w_tmp
                
        return w_min
    
    def poly_to_coefs(self, poly):
        coefs = poly.list() # https://ask.sagemath.org/question/26907/get-the-coefficients-from-the-polynomial/
        while len(coefs)<self.k:
            coefs.append(0)
        coefs.reverse()
        return coefs
    
    def decode_BW(self,U):
        y_inv = [self.y[i]^(-1) for i in range(self.n)]
        vec = vector(K, U)
        for i in range(n):
            vec[i] *= y_inv[i]
        M = Matrix(self.K, self.n, self.n + 1) 
        for i in range(self.n):
            for j in range(self.t + 1):
                M[i,j] = vec[i]*(self.x[i])^j
            for j in range(self.n - self.t):
                M[i,self.t+1+j] = -(self.x[i])^(j)
        #print(M)
        
        coefs = M.right_kernel().basis_matrix().rows()[0]
        R.<X> = PolynomialRing(self.K)
        A = 0
        B = 0
        tmp = 1
        for i in range(self.t + 1):
            A += coefs[i]*tmp
            tmp *= X
        tmp = 1
        for i in range(self.n - self.t):
            B += coefs[self.t + i + 1] * tmp
            tmp *= X
        r = B//A
        coefs = self.poly_to_coefs(r)

        return vector(self.K, coefs)
    
    def decode_BM(self, u):
        pol.<X> = PolynomialRing(self.K)
        points = []
        Pi = 1
        for i in range(self.n):
            points.append((self.x[i],u[i]*(self.y[i])^(-1)))
            Pi = Pi * (X - self.x[i])
        U = pol.lagrange_polynomial(points)
        #print(U)
        C0, A0, B0 = 1, 0 , Pi
        C1, A1, B1 = 0, 1 , U
        while B1.degree() >= self.n - self.t:
            B2 = B0 % B1
            Q = B0 // B1
            A2 = A0 - Q * A1
            C2 = C0 - Q * C1
            A0, B0, C0 = A1, B1, C1
            A1, B1, C1 = A2, B2, C2
        r = B1//A1
        
        coefs = self.poly_to_coefs(r)
        
        return vector(self.K, coefs)
    
    def decode_GuSu(self, u, radius, multiplicity):
        points = []
        l = ceil(multiplicity*(self.n - radius)/(self.k - 1)) - 1
        for i in range(self.n):
            points.append((self.x[i],u[i]*(self.y[i])^(-1)))
        Q = gs_interpolation_lee_osullivan(points, radius, (multiplicity, l), self.k - 1 )
        #F = alekhnovich_root_finder(Q, maxd = self.k - 1 )
        F = roth_ruckenstein_root_finder(Q, maxd = self.k - 1 )
        if len(F) == 0:
            print("Error, no code found")
            return 0
        if len(F) == 1:
            return self.poly_to_coefs(F[0])
        else :
            # search the nearest solution to u
            uu = vector(self.K, u)
            candidat = 0
            min_w = self.n
            for i in range(len(F)):
                coefs = self.poly_to_coefs(F[i])
                encoded = self.encode(coefs)
                encoded = vector(self.K, encoded)
                print(encoded)
                diff = (uu - encoded).hamming_weight()
                if diff < min_w :
                    candidat = i
                    min_w = diff
            return self.poly_to_coefs(F[candidat])

        
    def decode_Su(self, u, radius):
        return decode_GuSu(self, u, radius, 1)

In [2]:
def canal(x, nb_err, K):
    n = len(x)
    I = Subsets(n,nb_err).random_element()
    y = []
    for i in range(n):
        y.append(x[i])
        if i+1 in I:
            err = 0
            while err == 0:
                err = K.random_element()
            y[i] += err
    return vector(K,y)
 

# Test sur un GRS avec q=8, n = 6 k = 4 et y aléatoire

In [3]:
q = 256
n=50
k=30
K.<g> = GF(q)

y = [0]*n

for i in range(n):
    r = 0
    while r == 0:
        r = K.random_element()
    y[i] = r

print("Y : ", y)

grs = GRS(q, n, k, K, g, y)

mess = vector(K, [K.random_element() for i in range(k)])
print("Message : ", mess)
c = grs.encode(mess)
print("Encoded message :", c)
u = canal(c, grs.t, K)
print("Code received : ", u)
m = grs.decode_BW(u)
print("Message decoded :", m)
print("Well decoded :", m == mess)

    

Y :  [g^4 + g^3 + g^2 + g, g^5, g^7 + g^6 + g^4 + g^3 + 1, g^5 + g^2 + g + 1, g^7 + 1, g^6 + g^5 + g^2 + g + 1, g^7 + g^6 + g^5 + g^3 + g + 1, g^7 + g^4 + g^3 + 1, g^5 + g^3 + 1, g^7 + g^2, g^7 + g^5 + g^2, g^5 + g^4 + g^2 + 1, g^6 + g^5 + g^2 + g + 1, g^7 + g^4 + g^3 + 1, g^7 + g^6 + g^5 + g^4 + g^2 + g + 1, g^7 + g^6 + g^5 + g^4 + g^3 + g^2 + g + 1, g^7 + g^6 + g^5 + g^4 + g^3 + g + 1, g^5 + g^4, g^4 + g^2 + 1, g^4 + g, g^7 + g^6 + g^4 + g^2 + g + 1, g^6 + g^5 + g^2, g^5 + g^3 + g^2 + 1, g^7 + g^6 + g^4 + g^3 + 1, g^7 + g^6 + g^5 + g^4 + g^3 + g, g^6 + g^5 + g^3, g^7 + g^6 + g^4 + g^3, g^6 + g^5 + g^2 + g + 1, g^7 + g^6 + g^4 + g^3 + g^2, g^7 + g^5 + g + 1, g^6 + g^4 + g^3 + g^2 + 1, g^6 + g^5 + g^4 + 1, g^7 + g^5 + g^2 + g, g^6 + g^4 + g^2 + g + 1, g^7 + g^6 + g^5 + g^3 + 1, g^7 + g^6 + g^4 + g^3 + g^2 + 1, g^7 + g^5 + g^4 + g^3 + g + 1, g^7 + g^4 + g^2 + g, g^7 + g^5 + g^4 + g^3 + g + 1, g^7 + g^4 + g^3 + g, g^7 + g^5, g^7 + g^6 + g^4 + g^3 + g + 1, g^7 + g^6 + g^3 + g, g^6 + g^5 +

In [4]:
m = 5
base = 3
q = base^m
n=50
k=30
K.<g> = GF(q)

y = [0]*n

for i in range(n):
    r = 0
    while r == 0:
        r = K.random_element()
    y[i] = r

print("Y : ", y)

grs = GRS(q, n, k, K, g, y)

mess = vector(K, [K.random_element() for i in range(k)])
print("Message : ", mess)
c = grs.encode(mess)
print("Encoded message :", c)
u = canal(c, grs.t, K)
print("Code received : ", u)
m = grs.decode_BM(u)
print("Message decoded :", m)
print("Well decoded :", m == mess)

    

Y :  [2*g^4 + 1, g^4 + 2*g^3 + g^2, 2*g^4 + g^3 + 1, g^3, 2*g^4 + g^2 + 2*g + 2, 2*g^4 + g^3 + g^2 + g + 1, 2*g^4 + 2*g^3 + g^2 + g + 2, g^4 + 2*g^3 + 2, g^3 + 2*g^2 + g + 1, 2*g^4 + 2*g^2 + g + 2, 2*g^2 + g + 2, g^4 + 2*g^3 + g^2 + 2, g^4 + 2*g^2 + 2, 2*g^4 + g^3 + 2*g^2 + 2, g^4 + g^2, g^3 + 2*g^2 + 2*g + 1, 2*g^4 + 2*g^3 + 2*g^2 + 2*g + 1, 2*g^3 + 2*g^2 + 2*g + 1, 2*g^4 + g^3 + 2*g + 1, g^4 + g^3 + 2*g^2 + 2, g^3 + 2*g, g^4 + 2*g^3 + 2*g^2 + g + 1, 2*g^4 + g^3 + g^2 + 2, 2*g^3 + g^2 + g + 2, 2*g^3 + g, 2*g^4 + 2*g^3 + 2*g^2 + g, 2*g^3 + g^2 + 2*g + 1, 2*g^3 + 2*g^2 + 2*g, 2*g^4 + 2*g^2 + 2*g, g^3 + 2*g + 1, g^4 + 2*g^3 + g^2 + 2*g, 2*g^4 + 2*g^2 + g, 2*g + 1, 2*g^2 + g, 2*g^3 + g, g^4 + g^2 + 2*g + 2, 2*g^4 + 2*g^3 + 2*g^2 + 2, 2*g^4 + g^3 + g^2, 2*g^4 + 2*g^3 + 2*g + 2, 2*g^4 + g^3 + g^2 + 2*g + 1, 2*g^4 + g^3 + 1, 2*g^4 + g^3 + g^2 + 2*g, g^3 + 2*g^2 + g + 2, g^4 + 2*g + 2, g^4 + g^2 + 2*g + 1, 2*g^2 + 2*g + 2, 2*g^4 + 2*g^2 + g + 1, g^2, 2*g^3 + g, 2*g^4 + g^2 + 2]
Message :  (g^

# Test Gurusuami Sudan


In [58]:
m = 3
base = 2
q = base^m
n = 6
k = 4
K.<g> = GF(q)

y = [0]*n
for i in range(n):
    random = 0
    while random == 0:
        random = K.random_element()
    y[i] = random

y = vector(K, y)
grs = GRS(q, n, k, K, g, y)

mess = vector(K, [K.random_element() for i in range(k)])
print("Message : ", mess)
c = grs.encode(mess)
print("Encoded message :", c)
u = canal(c, grs.t, K)
print("Code received : ", u)
m = grs.decode_GuSu( u, 2, n) 
m = vector(K, m)
print("Message decoded :", m)
print("Well decoded :", m == mess)


Message :  (g + 1, g + 1, g + 1, g)
Encoded message : (g^2 + 1, 1, 0, g^2 + 1, g + 1, g + 1)
Code received :  (g^2 + 1, 1, 0, g^2 + 1, g^2, g + 1)
Message decoded : (g + 1, g + 1, g + 1, g)
Well decoded : True
